# Project Explanation

## Overview
This notebook is part of a project focused on predicting income levels based on a dataset obtained from the U.S. Census Bureau. The dataset has been preprocessed and cleaned for the purpose of a binary classification task, where the goal is to determine whether an individual's income is above or below k 0,0for further analysis and improvements.


In [42]:
# setup the environment
!conda activate income-pred

In [43]:
# import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTENC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils import resample
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV

In [44]:
def drop_num_outliers(df_num, num_features,  multiplier):
    """Drop outliers from a DataFrame based on the interquartile range (IQR) method.

    :param df_num: (pd.DataFrame) Input DataFrame containing numerical data.
    :param num_features: list of strings as column values
    :param multiplier: (float) Multiplier for determining the outlier threshold.
    :return: DataFrame with outliers removed.
    """
    for column in num_features:
        Q1 = df_num[column].quantile(0.25)
        Q3 = df_num[column].quantile(0.75)
        IQR = Q3 - Q1
        
        outliers = (df_num[column] < Q1 - multiplier * IQR) | (df_num[column] > Q3 + multiplier * IQR)
    
        print(f"number of Outliers of column {column} is {df_num[outliers].shape[0]}")
        df_num_no_outlier = df_num[~outliers]
    return df_num_no_outlier

def drop_cat_outliers(df, categorical_features, percentage=200):
    """For categorical data, we simply check the frequency of each category. 
    
    Categories with significantly lower frequencies than others may be considered outliers.
    This is set through percentage value

    :param df_num: (pd.DataFrame) Input DataFrame containing numerical data.
    :param categorical_features: list of strings as column values
    :param percentage: (int) percentage for determining the outlier threshold.
    :return: DataFrame with outliers removed.
    """
    for column in categorical_features:
        category_counts = df[column].value_counts()
    
        threshold = category_counts.max()//percentage
        outliers = (category_counts < threshold)
        outliers = outliers[outliers==True].index.values
        print(f"number of Outliers of column {column} is {outliers[outliers==True].sum()}")
        df = df[df[column].isin(outliers) == False]
    return df


## Data Preprocessing
Following the EDA, the notebook includes data preprocessing steps such as:
- DROP unknown values
- DROP Duplicates
- Separate numerical and categorical features
- Drop numerical and categorical outliers
- Encode categorical features

In [54]:
def preprocessing(df, numerical_features, cols2drop, unknown_values):
    """ Preprocessing steps on dataset
    
    DROP unknown values
    DROP Duplicates
    Separate numerical and categorical features
    Drop numerical and categorical outliers
    
    :param df:(pd.DataFrame) Input DataFrame containing numerical data.
    :param numerical_features: list of strings as numerical features
    :param cols2drop: list of strings as columns to drop
    :param unknown_values: list of strings as unknown values
    :return: preprocessed dataset
    """
    df = df.map(lambda x: str(x).lower().strip())  # to match all entries
    
    # Drop unknown values
    df_clean = df.drop(columns=cols2drop) 
    df_clean = df_clean[~df_clean.isin(unknown_values).any(axis=1)]
    print(f"shape of dataframe after dropping unknown values {df_clean.shape}")
    
    # Drop Duplicates
    df_clean = df_clean.drop_duplicates()
    df_clean.reset_index(drop=True, inplace=True)
    print(f"shape of dataframe after dropping the duplicated rows is : {df_clean.shape}")
    
    # Separate numerical and categorical features
    categorical_features = list(set(df_clean.columns.tolist()) - set(numerical_features) - set(['income']))
    df_num = df_clean[numerical_features]
    df_num = df_num.apply(pd.to_numeric, errors='coerce')
    df_cat = df_clean[categorical_features]
    label = df_clean['income']
    df_clean = pd.concat([df_num, df_cat, label], axis=1 )
    print(f'df_clean shape is {df_clean.shape}')
    
    # Drop numerical and categorical outliers
    df_clean = drop_num_outliers(df_clean, numerical_features, multiplier=3)
    print(f"shape of dataframe after dropping numerical outliers {df_clean.shape}")
    df_clean = drop_cat_outliers(df_clean, categorical_features, percentage=200)
    print(f"shape of dataframe after dropping categorical outliers {df_clean.shape}")
    
    # reset index
    df_clean.reset_index(drop=True, inplace=True)
    
    return df_clean

In [55]:
def encoding(df_clean, categorical_features):
    """ Encodes the dataset given its categorical features as a list of strings

    :param df_clean: dataframe
    :param categorical_features: a list of categorical features as strings
    :return: the dataframe with categorical columns
    """
    label_encoder = LabelEncoder()
    for col_name in categorical_features:
        df_clean[col_name] = label_encoder.fit_transform(df_clean[col_name])
        
        # Print the encoding information for each categorical feature
        print(f"Encoding information for {col_name}:")
        print(f"Original classes: {label_encoder.classes_}")
        print(f"Encoded values: {list(range(len(label_encoder.classes_)))}")
        print()
        
    # separately encode label 
    df_clean['income'] = df_clean['income'].apply(lambda x: 0 if x == "-50000" else 1)
    df_clean['income'].value_counts()
    return df_clean

In [56]:
numerical_features = ['age', 'instance weight', 'num persons worked for employer', 'weeks worked in year']

cols2drop =['migration code-change in msa', 'migration code-change in reg', 'migration code-move within reg',
         'migration prev res in sunbelt', 'class of worker','enroll in edu inst last wk', 'major industry code',
         'major occupation code', 'member of a labor union', 'reason for unemployment',
         'region of previous residence', 'state of previous residence', 'live in this house 1 year ago', 
         'migration prev res in sunbelt', 'family members under 18', "fill inc questionnaire for veteran's admin"]
unknown_values = ["?", "not in universe"]

In [57]:
train_file_path = '../data/census_income_learn.csv'
df = pd.read_csv(train_file_path)
print('preprocessing started---------------------------------------------')
df_train = preprocessing(df, numerical_features, cols2drop, unknown_values)

print('encoding started--------------------------------------------')
df_train = encoding(df_train, categorical_features)
print(df_train.shape)

preprocessing started---------------------------------------------
shape of dataframe after dropping unknown values (191239, 27)
shape of dataframe after dropping the duplicated rows is : (186617, 27)
df_clean shape is (186617, 27)
number of Outliers of column age is 0
number of Outliers of column instance weight is 872
number of Outliers of column num persons worked for employer is 0
number of Outliers of column weeks worked in year is 0
shape of dataframe after dropping numerical outliers (186617, 27)
number of Outliers of column detailed household and family stat is 0
number of Outliers of column country of birth mother is 0
number of Outliers of column veterans benefits is 0
number of Outliers of column dividends from stocks is 0
number of Outliers of column own business or self employed is 0
number of Outliers of column citizenship is 0
number of Outliers of column race is 0
number of Outliers of column wage per hour is 0
number of Outliers of column education is 0
number of Outli


## Model Selection and Training
The notebook utilizes the RandomForestClassifier for the binary classification task. The following steps are covered:
- Train-test split of the dataset.
- Training the RandomForestClassifier.
- Evaluating the model's accuracy on the test set.

In [58]:
def downsample(X_train, y_train, coef):
    """Downsamples the majority class in the training data to balance the class distribution.

    :param X_train: (pd.DataFrame): The feature matrix of the training data.
    :param y_train: (pd.Series): The target labels of the training data.

    :return: Tuple containing downsampled feature matrix (pd.DataFrame) and downsampled target labels (pd.Series).
    """

    train_data = pd.concat([X_train, y_train], axis=1)

    minority_class = train_data[train_data['income'] == 1]
    majority_class = train_data[train_data['income'] == 0]
    print(f'minority class (1) shape {minority_class.shape}, majority(0) class shape {majority_class.shape}')

    majority_downsampled = majority_class.sample(n=len(minority_class)*coef, random_state=42)
    downsampled_train_data = pd.concat([majority_downsampled, minority_class])
    X_train_downsampled = downsampled_train_data.drop('income', axis=1)

    y_train_downsampled = downsampled_train_data['income']
    
    return X_train_downsampled, y_train_downsampled


In [60]:
# Train-Test Split
X = df_clean.drop(columns=["income"])
y = df_clean["income"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y, random_state=42)

smote_nc = SMOTENC(categorical_features=categorical_features, random_state=42)
X_resampled, y_resampled = smote_nc.fit_resample(X_train, y_train)
print('dataset balanced')
print('value counts after oversampling', pd.DataFrame(y_resampled).value_counts())

dataset balanced
value counts after oversampling income
0         123417
1         123417
Name: count, dtype: int64


In [61]:
model = LGBMClassifier(random_state=42)
model.fit(X_resampled, y_resampled)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 123417, number of negative: 123417
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.043682 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 858
[LightGBM] [Info] Number of data points in the train set: 246834, number of used features: 24
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
              precision    recall  f1-score   support

           0       0.99      0.93      0.96     13713
           1       0.21      0.57      0.30       429

    accuracy                           0.92     14142
   macro avg       0.60      0.75      0.63     14142
weighted avg       0.96      0.92      0.94     14142



In [ ]:
# Define the parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.001, 0.],
    'max_depth': [5, 10, 15],
    'subsample': [0.8, 0.9, 1.0]
    }

# Create an LGBM classifier
lgbm_model = lgb.LGBMClassifier(random_state=42)

# Use GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(lgbm_model, param_grid, scoring='accuracy', cv=3)
grid_search.fit(X_resampled, y_resampled)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Train the model with the best hyperparameters on the resampled data
best_model = lgb.LGBMClassifier(**best_params, random_state=42)
best_model.fit(X_resampled, y_resampled)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Evaluate the model
print("Best Hyperparameters:", best_params)
print("\nClassification Report:\n", classification_report(y_test, y_pred))


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 82278, number of negative: 82278
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.058701 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 858
[LightGBM] [Info] Number of data points in the train set: 164556, number of used features: 24
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accura

## Model Evaluation and Interpretation
Model evaluation metrics, including accuracy and classification report, are presented. Feature importance is analyzed to understand the significant predictors for income levels.

In [ ]:
# Feature Importance
feature_importances = model.feature_importances_
feature_importance_df = pd.DataFrame({"Feature": X.columns, "Importance": feature_importances})
feature_importance_df = feature_importance_df.sort_values(by="Importance", ascending=False)

# Plotting feature importances
plt.figure(figsize=(12, 8))
sns.barplot(x="Importance", y="Feature", data=feature_importance_df)
plt.title("Feature Importance")
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.show()

In [ ]:
eval_file_path = '../data/census_income_test.csv'
df_eval = pd.read_csv(eval_file_path)
print('preprocessing started')
df_eval = preprocessing(df_eval, numerical_features, cols2drop, unknown_values)

print('encoding started')
df_eval = encoding(df_eval, categorical_features)
X_eval = df_eval.drop(columns=["income"])
y_eval = df_eval["income"]

In [ ]:
y_pred_eval = best_model.predict(X_eval)
print("\nClassification Report:\n", classification_report(y_eval, y_pred_eval))

## Future Steps
The notebook concludes with considerations for future steps, including model deployment, potential improvements, and additional features for future iterations.

This project aims to provide insights into predicting income levels using machine learning techniques and serves as a foundation for further analysis and improvements.


Ethical considerations in AI, particularly when dealing with features like age and race, are crucial to ensure fairness, prevent bias, and promote responsible AI practices. Here are some key principles and techniques to address ethical concerns related to features like age and race in AI:

Fairness and Bias Mitigation:

Bias Assessment: Regularly assess your models for bias, particularly in sensitive attributes like race and age. Utilize metrics like disparate impact analysis to identify and rectify biased outcomes.
Fair Representations: Ensure that your dataset is representative of the diverse population. Make efforts to include a balanced representation of different age groups, races, and ethnicities.

In [ ]:
import shap

# Assuming 'model' is your trained RandomForestClassifier

# Create an explainer using SHAP
explainer = shap.TreeExplainer(model)

# Choose a specific instance for explanation (e.g., the first test instance)
instance_to_explain = X_test.iloc[0]

# Calculate SHAP values for the chosen instance
shap_values = explainer.shap_values(instance_to_explain)

# Display SHAP values for the chosen instance
print("Instance-level SHAP Values:")
print(shap_values)
